In [58]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *


In [59]:
E, nu = 50, 0.1
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)


In [60]:
L = 1
w = 0.2
left = Plane(Pnt(0,0,0), Vec(-1,0,0)).bc("left")
right = Plane(Pnt(L,0,0), Vec(1,0,0)).bc("right")
bottom = Plane(Pnt(0,0,0), Vec(0,-1,0)).bc("bottom")
top = Plane(Pnt(0,w,0), Vec(0,1,0)).bc("top")
front = Plane(Pnt(0,0,0), Vec(0,0,-1)).bc("front")
back = Plane(Pnt(0,0,w), Vec(0,0,1)).bc("back")
cube = left * right * bottom * top * front * back

geo = CSGeometry()
geo.Add(cube)
mesh = Mesh(geo.GenerateMesh(maxh=0.1))
Draw(mesh)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [61]:
ord_u = 2   
ord_phi = ord_u - 2

# H1-conforming finite element space
fesu = VectorH1(mesh, order=ord_u, dirichlet='left')
fesphi = L2(mesh, order=ord_phi)

fes = FESpace([fesu,fesphi])
# define trial- and test-functions
u, delta = fes.TrialFunction()
v, vphi = fes.TestFunction()
gfu_u = GridFunction(fesu)
gfu_u.Set((0,0,1))

psih = GridFunction(fesphi)
psih.Set(1)

uk = GridFunction(fesu)
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 
phi = CF(-0.1)    

max_PG_it = 100
max_iter_newton = 50

alpha = Parameter(1)
for k in range(1,max_PG_it):

    alpha.Set(2**k)
    print("Iteration of proximal galerkin: ", k)
    # assing uk, psik
    uk.vec.data = gfu_u.vec
    psik.vec.data = psih.vec
    for i in range(max_iter_newton):
                    # the bilinear-form 
        a = BilinearForm(fes)
        rho = 0.1
        grav = 9.81
        force = CoefficientFunction((0,0,-rho*grav))
        with TaskManager():

            # the stress tensor equation
            a += alpha * InnerProduct(Stress(Sym(Grad(u))), Sym(Grad(v))).Compile()*dx
            # proximal galerkin term
            a += delta * v[2] * dx
            a += u[2]*vphi*dx  
            if ord_phi == 0:
                a += -delta * exp(psih) * vphi * dx - eps * (delta * vphi * dx)
            else:
                a+= -delta * exp(psih) * vphi * dx - eps * (grad(delta) * grad(vphi) * dx)
            ##REVISAR c_\varepsilon 
            # the right hand side

            lf = LinearForm(fes)
            lf += alpha * force[2] * v[2] * dx 
            lf += (psik - psih)*v[2]*dx
            lf += (phi + exp(psih))*vphi*dx
        
            a.Assemble()
            lf.Assemble()
        
        # the solution field 
        gfu = GridFunction(fes)
        # r.data = lf.vec - a.mat * gfu.vec
        gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * lf.vec
        if Integrate((gfu.components[0]-gfu_u)**2, mesh) < 1e-8:
            print("newton tol")
            break
        gfu_u.vec.data = gfu.components[0].vec
        psih.vec.data = psih.vec + gfu.components[1].vec
    if Integrate((uk-gfu_u)**2, mesh) < 1e-8:
        print("PG tol")
        Draw(gfu.components[0], mesh, "displacement")
        break
    if k%10 == 0:
        Draw(gfu.components[0], mesh, "displacement")


Iteration of proximal galerkin:  1
newton tol
Iteration of proximal galerkin:  2
newton tol
Iteration of proximal galerkin:  3
newton tol
Iteration of proximal galerkin:  4
newton tol
Iteration of proximal galerkin:  5
newton tol
PG tol


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [63]:
import pickle
fil = open("gfu.pkl", "rb")
gfu2 = pickle.load(fil)
Draw(gfu2-gfu.components[0], mesh, "displacement")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [46]:
Draw(gfu2.components[0], mesh, "displacement")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [47]:
import numpy as np
9*(317.8)/19.0228

150.3564144079736